In [4]:
from gensim.models import KeyedVectors
import pandas as pd
from collections import Counter
import nltk
import re
import requests
import pickle
import json
import os

In [5]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


In [6]:
lasla = KeyedVectors.load_word2vec_format("/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec")
lasla.most_similar("seminarium")

[('carnarium', 0.8714821934700012),
 ('sudarium', 0.8584282398223877),
 ('uinarius', 0.8317396640777588),
 ('torcularium', 0.8269298672676086),
 ('armarium', 0.8191075325012207),
 ('milliarium', 0.8161847591400146),
 ('mortarium', 0.8139162063598633),
 ('asinarius', 0.7973706722259521),
 ('pomarium', 0.7952086329460144),
 ('olearius', 0.7895564436912537)]

In [7]:
lasla.most_similar("semen", topn=20)

[('sementis', 0.7769805788993835),
 ('germen', 0.65008145570755),
 ('legumen', 0.6492694020271301),
 ('materies', 0.62647545337677),
 ('sulcus', 0.6210276484489441),
 ('stercoro', 0.6130126118659973),
 ('genitalis', 0.6120802760124207),
 ('sumen', 0.6056757569313049),
 ('animans', 0.5975176095962524),
 ('elementa', 0.5954310297966003),
 ('seges', 0.5945314764976501),
 ('primordium', 0.5920283198356628),
 ('bipalium', 0.5892609357833862),
 ('omnigenus', 0.5877910256385803),
 ('ablaqueo', 0.5792651772499084),
 ('renascor', 0.573520839214325),
 ('enascor', 0.5693485736846924),
 ('abdomen', 0.5686165690422058),
 ('priuus', 0.5666219592094421),
 ('augmen', 0.5655485391616821)]

In [8]:
lasla.index_to_key[:10]

['</s>', 'sum', 'et', 'qui', 'in', 'non', 'hic', 'is', 'ut', 'ego']

In [9]:
operamaiora = KeyedVectors.load_word2vec_format("/srv/data/lila/opera-maiora-lemmas_skip_100.vec")
operamaiora.most_similar("anima")

[('intellectiuus', 0.7068129181861877),
 ('separo', 0.7005686163902283),
 ('sensifico', 0.6973943114280701),
 ('sensitiuus', 0.6771703362464905),
 ('corpus', 0.663750171661377),
 ('uegetabilis', 0.6309762597084045),
 ('unio', 0.6247175335884094),
 ('rationalis', 0.623648464679718),
 ('uegetatiuus', 0.618419885635376),
 ('corporeus', 0.60992830991745)]

In [10]:
operamaiora.most_similar("semen")

[('formatiuus', 0.7347741723060608),
 ('decidocaedo', 0.7292525768280029),
 ('sementis', 0.7260836362838745),
 ('decisio', 0.718953549861908),
 ('proauus', 0.708860456943512),
 ('menstruus', 0.7087300419807434),
 ('fetus', 0.7033671736717224),
 ('incubus', 0.6823228597640991),
 ('seminalis', 0.6802768111228943),
 ('spumosus', 0.6695918440818787)]

In [11]:
operamaiora_vocab = operamaiora.index_to_key
len(operamaiora_vocab)

6724

In [12]:
lasla_vocab = lasla.index_to_key
len(lasla_vocab)

11327

In [13]:
vocab_overlap = list(set(lasla_vocab) & set(operamaiora_vocab))
len(vocab_overlap)

3950

In [14]:
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"

In [15]:
id = os.listdir(source_path)[0]

In [16]:
f_sents_data = json.load(open(source_path + str(id), "rb"))
f_sents_data[:3]

[['604894',
  0,
  'Noua Acta Heluetica Physico-mathematico- Anatomico-botanico- Medica Tabulis Aeneis Illustrata Et In Usus Publicos Exarata.',
  [['Noua', 'nouus', 'ADJ', [0, 4]],
   ['Acta', '', 'VERB', [5, 9]],
   ['Heluetica', 'helueticus', 'ADJ', [10, 19]],
   ['Physico', 'physicus', 'PROPN', [20, 27]],
   ['-', '-', 'PUNCT', [27, 28]],
   ['mathematico-', '', 'ADJ', [28, 40]],
   ['Anatomico', 'anatomicus', 'PROPN', [41, 50]],
   ['-', '-', 'PUNCT', [50, 51]],
   ['botanico-', '', 'PRON', [51, 60]],
   ['Medica', 'medica', 'PROPN', [61, 67]],
   ['Tabulis', 'tabula', 'NOUN', [68, 75]],
   ['Aeneis', 'aeneus', 'ADJ', [76, 82]],
   ['Illustrata', 'illustro', 'VERB', [83, 93]],
   ['Et', 'et', 'CCONJ', [94, 96]],
   ['In', 'in', 'ADP', [97, 99]],
   ['Usus', 'usus', 'NOUN', [100, 104]],
   ['Publicos', 'publicus', 'PROPN', [105, 113]],
   ['Exarata', 'exaro', 'VERB', [114, 121]],
   ['.', '.', 'PUNCT', [121, 122]]]],
 ['604894',
  1,
  'Uolumen I. Basileae, Typis & Sumptibus Joanni

In [17]:
def get_filtered_counts_by_ids(ids, lowercase=True):
    vocab_counter = Counter() # initiate an empty counter of words - we will feed it iterarively on the go by all words we meet
    for id in ids: # for each work ID from our subset of IDs
        # based on the ID, open the file containing the lemmatized sentences
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n = len(f_sents_data)
        tokens_n = 0
        lemmata = []
        for sent_data in f_sents_data:
            tokens_n += len(sent_data[3])
            sent_lemmata = [t[1] for t in sent_data[3] if t[2] in ["NOUN", "VERB", "ADJ", "PROPN"]]
            sent_lemmata = [re.sub(r"\W*|\d*", "", t) for t in sent_lemmata]
            sent_lemmata = [l for l in sent_lemmata if len(l) > 1]
            lemmata.extend(sent_lemmata)
        if lowercase:
            lemmata = [l.lower() for l in lemmata]
        #overview_dict[filename.partition(".")[0]] = {"sents_n" : sents_n, "tokens_n" : tokens_n} 
        word_counts = dict(nltk.FreqDist(lemmata).most_common())
        vocab_counter.update(word_counts)
    return vocab_counter

In [18]:
ids = metadata_table_long[metadata_table_long["file_year"].between(1600, 1610)]["id"]
vocab = get_filtered_counts_by_ids(ids)

In [19]:
list(vocab.items())[:100]

[('amor', 436),
 ('animus', 907),
 ('do', 2857),
 ('possum', 5880),
 ('habeo', 6613),
 ('google', 697),
 ('uideo', 4251),
 ('ualeo', 441),
 ('facio', 5891),
 ('uita', 1278),
 ('dico', 6654),
 ('deus', 1036),
 ('fero', 1987),
 ('lux', 1082),
 ('oculus', 2649),
 ('nomen', 2136),
 ('dies', 5150),
 ('mens', 601),
 ('os', 6732),
 ('musa', 193),
 ('acidalius', 102),
 ('bonus', 1010),
 ('uis', 2453),
 ('fides', 612),
 ('janus', 98),
 ('uolo', 1823),
 ('nouus', 952),
 ('uiuo', 677),
 ('dolor', 1055),
 ('pectus', 599),
 ('manus', 1347),
 ('ingenium', 469),
 ('tempus', 4281),
 ('homo', 2138),
 ('amicus', 413),
 ('magnus', 3618),
 ('carmen', 185),
 ('cura', 397),
 ('orbis', 1154),
 ('ago', 1172),
 ('felix', 448),
 ('flos', 527),
 ('uenio', 1317),
 ('amo', 274),
 ('diuus', 159),
 ('cor', 2420),
 ('mors', 719),
 ('caput', 5098),
 ('eo', 368),
 ('numen', 111),
 ('annus', 5028),
 ('spes', 229),
 ('capio', 587),
 ('dignus', 426),
 ('credo', 611),
 ('puer', 442),
 ('res', 3639),
 ('ars', 1282),
 ('hono

In [20]:
# generate vocabularies for each half-century under scrutiny
periods = [(1501, 1550),
           (1551, 1600),
           (1601, 1650),
           (1651, 1700)]
periods_vocabs = []
full_vocabs = []
for per in periods:
    ids = metadata_table_long[metadata_table_long["file_year"].between(per[0], per[1])]["id"] # use the metadata table to extract IDs of all works falling into this period
    vocab = get_filtered_counts_by_ids(ids)
    periods_vocabs.append(vocab)

In [21]:
list(periods_vocabs[0].items())[:10]

[('aqua', 11747),
 ('terra', 14371),
 ('lapis', 4154),
 ('metallum', 1984),
 ('color', 6554),
 ('res', 29069),
 ('dico', 43255),
 ('possum', 25029),
 ('facio', 31814),
 ('habeo', 31690)]

In [47]:
# only 2,000 most common words for each period...
topn = 2000
vocabs_topn_lists =[list(vocab.keys())[:topn] for vocab in periods_vocabs]

In [48]:
# build one list of words, including all from these four sets, i.e. their union
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists])
len(vocab_union)

4997

In [49]:
# add there also the words appearing both in lasla and operamaiora
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists] + [set(vocab_overlap)])
len(vocab_union)

6643

In [50]:
vocab_union_df = pd.DataFrame(vocab_union, columns = ["word"])
vocab_union_df

,word
0,nouus
1,thracia
2,labrum
3,mercator
4,honestum
...,...
6638,obsto
6639,multus
6640,pontus
6641,legatus


In [51]:
# add the counts of these words from the vocabularies
def find_in_vocab(word, vocab):
    try:
        return vocab[word]
    except:
        return 0
for vocab, per in zip(periods_vocabs, [str(tup[0])+"-"+str(tup[1]) for tup in periods]):
    vocab_union_df[per] = vocab_union_df["word"].apply(lambda x: find_in_vocab(x, vocab))

In [52]:
# calculate the mean value for sorting
vocab_union_df["mean"] = vocab_union_df[vocab_union_df.columns[1:]].mean(axis=1)
vocab_union_df.sort_values("mean", ascending=False, inplace=True)
vocab_union_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean
1048,dico,43255,122549,42457,72224,70121.25
5667,pars,33225,93059,44337,52385,55751.50
631,possum,25029,82700,38391,76841,55740.25
4359,habeo,31690,89199,39443,59770,55025.50
144,facio,31814,98524,36944,51438,54680.00
1072,liber,39559,65900,38496,53100,49263.75
5098,res,29069,70669,28291,51565,44898.50
4640,uideo,21536,59099,26372,50503,39377.50
4861,locus,23102,56782,31421,43680,38746.25
924,corpus,16979,63171,22871,31938,33739.75


In [53]:
vocab_union_df.tail(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean
3603,obuiam,0,0,0,0,0.0
3604,piget,0,0,0,0,0.0
3900,qualiscumque,0,0,0,0,0.0
6037,antea,0,0,0,0,0.0
496,instans,0,0,0,0,0.0
5796,paene,0,0,0,0,0.0
1096,saltem,0,0,0,0,0.0
2018,</s>,0,0,0,0,0.0
25,sicubi,0,0,0,0,0.0
6604,o,0,0,0,0,0.0


In [54]:
# Filter only for words appearing at least 5 times in each of the subcorpora
filtered_vocab_df = vocab_union_df[
    (vocab_union_df['1501-1550'] >= 5) &
    (vocab_union_df['1551-1600'] >= 5) &
    (vocab_union_df['1601-1650'] >= 5) &
    (vocab_union_df['1651-1700'] >= 5)
    ]

In [55]:
len(filtered_vocab_df)

6005

In [56]:
filtered_vocab_df.head(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean
1048,dico,43255,122549,42457,72224,70121.25
5667,pars,33225,93059,44337,52385,55751.50
631,possum,25029,82700,38391,76841,55740.25
4359,habeo,31690,89199,39443,59770,55025.50
144,facio,31814,98524,36944,51438,54680.00
1072,liber,39559,65900,38496,53100,49263.75
5098,res,29069,70669,28291,51565,44898.50
4640,uideo,21536,59099,26372,50503,39377.50
4861,locus,23102,56782,31421,43680,38746.25
924,corpus,16979,63171,22871,31938,33739.75


In [57]:
filtered_vocab_df.tail(10)


,word,1501-1550,1551-1600,1601-1650,1651-1700,mean
3628,innuptus,7,7,16,11,10.25
4184,peiero,6,6,12,15,9.75
1239,catechumenus,6,17,6,9,9.50
5111,ipse,7,12,9,9,9.25
732,monumen,6,14,10,7,9.25
2659,repudium,8,7,7,12,8.50
2816,iste,5,10,13,5,8.25
5853,charon,12,8,7,6,8.25
3573,semper,5,6,6,7,6.00
897,inornatus,5,7,6,5,5.75


In [58]:
filtered_vocab_df["in_lila_embeddings"] = filtered_vocab_df["word"].apply(lambda x: x in vocab_overlap)

/tmp/ipykernel_696720/3542024209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_vocab_df["in_lila_embeddings"] = filtered_vocab_df["word"].apply(lambda x: x in vocab_overlap)


In [59]:
filtered_vocab_df["in_lasla"] = filtered_vocab_df["word"].apply(lambda x: x in lasla_vocab)
filtered_vocab_df["in_operamaiora"] = filtered_vocab_df["word"].apply(lambda x: x in operamaiora_vocab)

/tmp/ipykernel_696720/3123464796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_vocab_df["in_lasla"] = filtered_vocab_df["word"].apply(lambda x: x in lasla_vocab)
/tmp/ipykernel_696720/3123464796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_vocab_df["in_operamaiora"] = filtered_vocab_df["word"].apply(lambda x: x in operamaiora_vocab)


In [60]:
filtered_vocab_df.head(100)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora
1048,dico,43255,122549,42457,72224,70121.25,True,True,True
5667,pars,33225,93059,44337,52385,55751.50,True,True,True
631,possum,25029,82700,38391,76841,55740.25,True,True,True
4359,habeo,31690,89199,39443,59770,55025.50,True,True,True
144,facio,31814,98524,36944,51438,54680.00,True,True,True
...,...,...,...,...,...,...,...,...,...
4136,bonus,5865,16300,5389,9261,9203.75,True,True,True
2327,proprius,3576,17662,7350,8199,9196.75,True,True,True
5064,singulus,6167,13325,7083,9816,9097.75,True,True,True
1177,principium,4584,17735,6334,7463,9029.00,True,True,True


In [61]:
filtered_vocab_df["in_lasla"].sum()

4820

In [62]:
filtered_vocab_df["in_operamaiora"].sum()


3962

In [63]:
filtered_vocab_df["in_lila_embeddings"].sum()


3708

# Add simple english translation

In [41]:
# simple latin API:
# https://www.latin-is-simple.com/api/

In [42]:
target = "equus"
base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
resp_json = requests.get(base_url.format(target)).json()
resp_json

[{'id': 63,
  'intern_type': 'noun',
  'short_name': 'equus',
  'full_name': 'equus, equi [m.] O',
  'type': {'name': 'noun', 'label': 'Noun'},
  'translations_unstructured': {'en': 'horse', 'de': 'Pferd'},
  'url': 'https://www.latin-is-simple.com/en/vocabulary/noun/63/'}]

In [43]:
resp_json[0]["translations_unstructured"]["en"]

'horse'

In [44]:
def simple_translation(target):
    try:
        base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
        resp_json = requests.get(base_url.format(target)).json()
        transl = resp_json[0]["translations_unstructured"]["en"]
        if transl == "still in translation":
            transl = ""
        return transl
    except:
        return ""

In [45]:
%%time
filtered_vocab_df["transl"] = filtered_vocab_df["word"].apply(simple_translation)

CPU times: user 53.2 s, sys: 4.17 s, total: 57.4 s
Wall time: 30min 22s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [46]:
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
986,dico,43255,122549,42457,72224,70121.25,True,True,True,"say, call, tell"
11948,pars,33225,93059,44337,52385,55751.50,True,True,True,part
7039,possum,25029,82700,38391,76841,55740.25,True,True,True,"be able, can"
10696,habeo,31690,89199,39443,59770,55025.50,True,True,True,"have, hold, possess, consider, think"
141,facio,31814,98524,36944,51438,54680.00,True,True,True,"do, make, handle"
1007,liber,39559,65900,38496,53100,49263.75,True,True,True,"book, volume, inner bark of a tree, book, book"
4926,res,29069,70669,28291,51565,44898.50,True,True,True,suddenly
4477,uideo,21536,59099,26372,50503,39377.50,True,True,True,
11185,locus,23102,56782,31421,43680,38746.25,True,True,True,"place, location"
878,corpus,16979,63171,22871,31938,33739.75,True,True,True,"body, corpus"


In [75]:
filtered_vocab_df.to_json("../data/filtered_vocab_df.json")

In [2]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json

In [76]:
# read the old one just for comparison
filtered_vocab_df_OLD = pd.read_json("../data/filtered_vocab_df_OLD.json")
filtered_vocab_df_OLD.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
5018,dico,42831,120825,41832,72221,69427.25,True,True,True,"say, call, tell"
6155,omnis,33152,108530,46009,66387,63519.50,True,True,True,
2515,facio,33323,109486,40134,53953,59224.00,True,True,True,"do, make, handle"
4040,pars,32591,90667,43593,51249,54525.00,True,True,True,part
6769,habeo,30884,87225,38386,57791,53571.50,True,True,True,"have, hold, possess, consider, think"
364,possum,23869,79572,36898,72892,53307.75,True,True,True,"be able, can"
897,res,27869,68790,27473,49014,43286.50,True,True,True,suddenly
4810,suus,23716,58975,33267,52441,42099.75,True,True,True,"his own, her own, its own, their own"
2392,uideo,21389,60143,28355,54288,41043.75,True,True,True,
1790,magnus,23446,61213,29708,39211,38394.50,True,True,True,"big, great"
